In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')
#добавляю для себя, так у меня работают пакеты!

In [2]:
import pandas as pd
from tqdm import tqdm
import os
import re
from nltk import trigrams
from collections import Counter
import pymorphy2 # Лемматизация

## Скачиваем тексты wiki ##

In [20]:
import wikipedia

In [21]:
wikipedia.set_lang("ru")

In [28]:
random_pages_names = wikipedia.random(pages=500)

In [4]:
path_train_texts = 'C:/Users/Анна/YandexDisk/spell-check/wikiru/'

In [3]:
for random_name in tqdm(random_pages_names):
    page_text = []
    random_name = wikipedia.search(random_name)
    try:
        saved_page = wikipedia.page(random_name[0])
    except (wikipedia.exceptions.DisambiguationError,
            wikipedia.exceptions.PageError) as e:
        print("Error: {0}".format(e))
        continue
    try:
        page_text.append(saved_page.title)
        page_text.append(saved_page.summary)
        page_text.append(saved_page.content)
    except wikipedia.exceptions.DisambiguationError as e:
        print("Error: {0}".format(e))
        continue
             
   
    page_text = ' '.join(page_text)

    file_name = (path_train_texts+(saved_page.title) + '.txt')
    try:
        with open(file_name, 'w', encoding = 'utf-8') as fl:
            fl.write(page_text)
    except FileNotFoundError:
        continue

'for random_name in tqdm(random_pages_names):\n    page_text = []\n    random_name = wikipedia.search(random_name)\n    try:\n        saved_page = wikipedia.page(random_name[0])\n    except (wikipedia.exceptions.DisambiguationError,\n            wikipedia.exceptions.PageError) as e:\n        print("Error: {0}".format(e))\n        continue\n    try:\n        page_text.append(saved_page.title)\n        page_text.append(saved_page.summary)\n        page_text.append(saved_page.content)\n    except wikipedia.exceptions.DisambiguationError as e:\n        print("Error: {0}".format(e))\n        continue\n             \n   \n    page_text = \' \'.join(page_text)\n\n    file_name = (path_train_texts+(saved_page.title) + \'.txt\')\n    try:\n        with open(file_name, \'w\', encoding = \'utf-8\') as fl:\n            fl.write(page_text)\n    except FileNotFoundError:\n        continue'

In [9]:
def slurp(path):
    with open(path, 'r', encoding = 'utf-8') as file_object:
        return file_object.read()

## Препроцессинг ##

In [3]:
def preprocess(texts):
    preprocessed_texts_list = []
    for text in texts:
        text = re.sub(r'[A-Za-z]', '', text)
        text = re.sub(r'[А-Я]{2,}', '', text)#убираем аббревиатуры
        lower_text = text.lower()#приводим к нижнему регистру, заменяем тире, чтобы потом оставить дефисы в словах
        #lower_text = re.sub(r'(\d)+-(\d)+','', lower_text)#удаление цифр типа 1999-2001
        del_new_line = re.sub(r'\n', '', lower_text)
        no_ref_text = re.sub(r'==литература(.)+|==источники(.)+', '', del_new_line)#удаляем все, что после рубрики "Сссылки"
        no_num_text = re.sub(r'\d', '', no_ref_text)#удаляем цифры
        extracted_text = re.findall(r'[а-яё]+', no_num_text)
        #разделяем на токены - последовательности букв с дефисами
        extracted_text = ' '.join(extracted_text)
        preprocessed_texts_list.append(extracted_text)
    return preprocessed_texts_list

In [4]:
train_texts_list = []
for root, dirs, files in os.walk(path_train_texts):
    for file_name in tqdm(files):
        input_path = path_train_texts + file_name
        text = slurp(input_path)
        train_texts_list.append(text)
preprocessed_train_texts = preprocess(train_texts_list)    

NameError: name 'path_train_texts' is not defined

In [81]:
preprocessed_train_texts[14]

'число одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя свойства цифры первых пяти неотрицательных степеней числа являются элементами треугольника паскаля эндекахорон англ абстрактный правильный четырёхмерный политоп ячеек которого представляют собой полуикосаэдры число бога кубика рубика в метрике равно геометрия существует развёрток куба образующих подмножество гексамино и развёрток октаэдра образующих подмножество октиамондов из гексамино шестиклеточных полимино ровно являются чётными то есть при шахматной раскраске содержат чётное число светлых и чётное число тёмных квадратов дэвид а кларнер обнаружил 

In [82]:
train_wiki = pd.DataFrame(data = preprocessed_train_texts, columns=['wiki_texts'])

In [83]:
train_wiki.to_csv('train_wiki_2.csv', index = False, encoding = 'utf-8')

## Обучающий корпус - тексты Википедии ##

In [4]:
train_wiki = pd.read_csv('/home/nst/mount/data/share/yd/comp_ling/spell-check/train_wiki_2.csv')

In [4]:
train_wiki = pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_wiki_2.csv')

In [7]:
train_wiki.head()

,wiki_texts
0,долби долби лат астероид относящийся к группе ...
1,адрия адрия лат довольно крупный астероид из г...
2,гиппас гиппас лат др греч типичный троянский а...
3,гоголь гоголь лат типичный астероид главного п...
4,капрера капрера итал астероид главного пояса п...


## Обучающая выборка с ошибками

In [5]:
train = slurp('/home/nst/mount/data/share/yd/comp_ling/spell-check/training.txt')

In [10]:
train = slurp('C:/Users/Анна/YandexDisk/spell-check/training.txt')

In [6]:
train = train.split('\n\n')

In [7]:
train_df = pd.DataFrame(data = train, columns = ['train'])

In [8]:
train_df['wrong'], train_df['right'] = train_df['train'].str.split('\n',1).str

In [9]:
train_df.pop('train');

In [11]:
train_df.to_csv('train_df.csv', index = False, encoding="utf-8")

In [12]:
train_df = pd.read_csv('/home/nst/mount/data/linguistics_hse/comp_ling/spell_check/anyala_compling_repo/CompLing/spell_reading/train_df.csv')

In [11]:
train_df =  pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_df.csv') ;

In [12]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,"Вальс в 5 па ( внимание, будет под другую музы...",Вальс в 5 па внимание будет под другую музыку ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


In [13]:
#уберем знаки препинания из обучающей выборки
train_df.wrong = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in train_df.wrong]
train_df.right = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in train_df.right]

In [15]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


## Импорт словарей ##

### rucorpora ###

In [66]:
rucorpora_dict = slurp('/home/nst/mount/data/share/yd/comp_ling/spell-check/dict.opcorpora.txt')

In [14]:
rucorpora_dict = slurp('C:/Users/Анна/YandexDisk/spell-check/dict.opcorpora.txt')

In [15]:
rucorpora_dict = re.findall(r'[А-ЯЁ]+', rucorpora_dict)
rucorpora_dict = ' '.join(rucorpora_dict)
rucorpora_dict = rucorpora_dict.lower()
#rucorpora_dict = rucorpora_dict.split()

In [16]:
rucorpora_dict[:20]

'ёж ежа ежу ежа ежом '

## Строим символьные триграммы по тексту ##

In [21]:
def get_trigrams(text): #строим триграммы по тексту
    trigrams_list = []
    for word in text.split():
        trigram = [''.join(char) for char in list(trigrams(word))]
        if trigram:
            trigrams_list.append(' '.join(trigram))
        else:
            trigrams_list.append(word)   
   #tr = [' '.join([''.join(chars) for chars in list(trigrams(word))]) for word in text.split()]
    return trigrams_list

In [90]:
print(train_df.wrong[0])
print(get_trigrams(train_df.wrong[0]))
print(' '.join(get_trigrams(train_df.wrong[0])).split())

тоесть если дамой то только в том случае если потеряю работу в сентябре
['тое оес ест сть', 'есл сли', 'дам амо мой', 'то', 'тол оль льк ько', 'в', 'том', 'слу луч уча чае', 'есл сли', 'пот оте тер еря ряю', 'раб або бот оту', 'в', 'сен ент нтя тяб ябр бре']
['тое', 'оес', 'ест', 'сть', 'есл', 'сли', 'дам', 'амо', 'мой', 'то', 'тол', 'оль', 'льк', 'ько', 'в', 'том', 'слу', 'луч', 'уча', 'чае', 'есл', 'сли', 'пот', 'оте', 'тер', 'еря', 'ряю', 'раб', 'або', 'бот', 'оту', 'в', 'сен', 'ент', 'нтя', 'тяб', 'ябр', 'бре']


##  Вероятности символьных триграмм в корпусе

In [20]:
def count_trigrams(corpus): #получаем вероятности триграмм в корпусе
    corpus_trigrams = []
    for text in corpus:
        tr = ' '.join(get_trigrams(text))
        corpus_trigrams.extend(tr.split())
    counts = dict(Counter(corpus_trigrams))
    for key, value in counts.items():
        counts[key] = value/len(corpus_trigrams)
    print(len(corpus_trigrams))    
    return counts

### Trigram wiki model

In [22]:
%%time
trigram_wiki_model = count_trigrams(train_wiki.wiki_texts)
#обучили 3-граммную модель на корпусе для получения вероятностей 3-грамм
#выводится общее число триграм в корпусе (c повторениями)

8297222
CPU times: user 16.2 s, sys: 432 ms, total: 16.7 s
Wall time: 16.9 s


In [32]:
trigram_wiki_model.get('есл') #вероятность 3-граммы "есл"

0.0001628255818634237

In [33]:
trigram_wiki_model.get('зрх') #такой 3-граммы нет, модель не возвращает ничего

In [34]:
len(trigram_wiki_model)

13679

In [35]:
10/8297222 #какой порог взять для проверки слова по словарю?

1.2052226636818925e-06

In [194]:
#trigram_wiki_model.items()

## Поиск ошибок в текстах ##

In [36]:
def search_errors(text): #поиск ошибок в тексте
    errors = []
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    #print(trigram_list)
    for i, trigrams_string in enumerate(trigram_list):
        for trigram in trigrams_string.split():
            #print(trigram_model.get(tr))
            if not(trigram_wiki_model.get(trigram)) or (trigram_wiki_model.get(trigram) < (10/8297222)):
                #print(trigram)
                #print(text.split()[i])
                #if morph.parse(text.split()[i])[0].normal_form not in rucorpora_dict:
                if text.split()[i] not in rucorpora_dict:
                    errors.append(text.split()[i])
    return list(set(errors))

In [37]:
search_errors(train_df.wrong[10])

['самыи', 'своеи']

In [38]:
wiki_errors = []
for text in tqdm(train_df.wrong):
    wiki_errors.extend(search_errors(text))

100%|██████████| 3000/3000 [00:50<00:00, 59.60it/s]


In [39]:
len(wiki_errors)

556

In [40]:
print(wiki_errors[:100])

['такойже', 'недюженные', 'пейзажы', 'самыи', 'своеи', 'щастье', 'никоглда', 'отличаються', 'расжился', 'подощрительно', 'преподше', 'замерзнуть', 'можнго', 'првязанность', 'любю', 'дабаа', 'йпри', 'поведйте', 'воопще', 'нмного', 'черезчур', 'неопределнное', 'обчным', 'всмысле', 'отношуся', 'мущщину', 'мачовсе', 'божеупаси', 'вопщем', 'болезньматки', 'ндфл', 'теща', 'лучшым', 'кагбэ', 'рыпки', 'каааак', 'тогдя', 'совршенно', 'нащщот', 'дефачка', 'оооочень', 'оплаьа', 'мужык', 'кефирчиком', 'обяснил', 'проблемнная', 'внутрениий', 'элкетроинструменты', 'всеже', 'хочеца', 'блетчли', 'такскаать', 'воопщим', 'разделеннный', 'расщифровать', 'прощаите', 'реклма', 'очеь', 'ооочень', 'хваостаюсь', 'маленбких', 'саиые', 'дежурцве', 'хотябы', 'наконецто', 'каааак', 'оооочень', 'щастье', 'собссно', 'продкты', 'удеается', 'бплгодаря', 'сногсшибальной', 'блекнет', 'посдравляю', 'ооочень', 'сеъли', 'подргрузился', 'всвязи', 'прошедшых', 'всмысле', 'вооот', 'обязаательно', 'погкуляем', 'удалитъ', 'мор

## Генерация кандидатов

In [17]:
def get_all_candidates(word):
    letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    cursor = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    delete = [L + R[1:] for (L,R) in cursor if R]
    insert = [L + l + R for l in letters for (L,R) in cursor]
    replace = [L + l + R[1:] for l in letters for (L,R) in cursor if R]
    transpose = [L + R[1] + R[0] + R[2:] for (L,R) in cursor if len(R)>1]
    return set(delete + insert + replace + transpose)

In [20]:
def kill_candidates(error_word):
    candidates = []
    #kills = []
    for word in get_all_candidates(error_word):
        if keyword_processor.extract_keywords(word) == [word]:
            candidates.append(word)
    return candidates

In [43]:
print(len(get_all_candidates('пейзажы')))

495


In [44]:
%%time
print(kill_candidates('пейзажы'))

['пейзаж', 'пейзаже', 'пейзажи', 'пейзажа', 'пейзажу', 'пейзажны', 'пейзаны']
CPU times: user 23.8 s, sys: 16 ms, total: 23.8 s
Wall time: 24.1 s


## Вероятности на словарных биграммах nltk

In [21]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [27]:
wiki_texts = train_wiki.wiki_texts
wiki_tokens = [word for text in wiki_texts for word in text.split()]
print(wiki_tokens[10:20])

['марса', 'он', 'был', 'открыт', 'декабря', 'года', 'американским', 'астрономом', 'любителем', 'чарльзом']


In [29]:
wiki_freq = nltk.FreqDist(wiki_tokens) #частотный словарь
#wiki_freq = Counter(wiki_tokens)
#wiki_freq

In [30]:
wiki_cfreq = nltk.ConditionalFreqDist(nltk.bigrams(wiki_tokens)) #словарь словарей для условных частот
#wiki_cfreq['и']

In [31]:
cprob = nltk.ConditionalProbDist(wiki_cfreq, nltk.MLEProbDist) #словарь условных вероятностей

print('p(русский язык) = %1.4f' %cprob['русский'].prob('язык'))
print('p(в доме) = %1.4f' %cprob['в'].prob('доме'))
print('p(солнечной системе) = %1.4f' %cprob['солнечной'].prob('системе'))

p(русский язык) = 0.1279
p(в доме) = 0.0011
p(солнечной системе) = 0.2558


In [32]:
def select_best_candidate(candidates, left_context, right_context):
    s = sum(dict(wiki_freq).values())
    #cand_prob = dict(wiki_freq).get(cand)/s #априорная вероятность кандидата
    
    #считаем условные вероятности в контексте
    if left_context: #если есть слово слева от ошибочного
        if right_context: #если есть слово справа
            cand_prob = [(cand, cprob[left_context].prob(cand) * 
                        cprob[cand].prob(right_context)) 
                         for cand in candidates]
        else:
            cand_prob = [(cand, cprob[left_context].prob(cand))
                         for cand in candidates]
            
    else:
        cand_prob = [(cand, cprob[cand].prob(right_context)) for cand in candidates]
        
    cand_rate = sorted(cand_prob, key = lambda column:column[1], reverse = True) #сортируем по вероятности 
    
    if not(cand_rate[0][1]):
        cand_aprior = [(cand, dict(wiki_freq).get(cand)/s) 
                       if dict(wiki_freq).get(cand) else (cand, 0) 
                       for cand in candidates]
        cand_rate = sorted(cand_aprior, key = lambda column:column[1], reverse = True)
        
    return cand_rate[0][0], cand_rate[0][1]

In [52]:
for text in train_df.wrong:
    if 'пейзажы' in text:
        print(text)

пейзажы за окном какие то незнакомые


In [53]:
train_df[train_df.wrong == 'пейзажы за окном какие то незнакомые']

,wrong,right
9,пейзажы за окном какие то незнакомые,пейзажи за окном какие то незнакомые


In [54]:
dict(wiki_freq).get('пейзажи')

12

In [208]:
select_best_candidate(['пейзаж', 'пейзажа', 'пейзажны', 'пейзаны', 'пейзаже', 'пейзажу', 'пейзажи'], '', 'за')

('пейзажи', 6.612932912897754e-06)

## Алгоритм спелл-чек

1. Поиск ошибок в тексте
2. Генерация кандидатов
3. Убийство кандидатов: если список слов оказывается пустым после проверки по словарю, возвращаем искомое слово (на случай, если его вдруг не оказалось в нашем словаре)
4. Считаем вероятность кандидата в контексте. Вставляем в текст кандидата с наибольшей вероятностью.
5. Возвращаем новый исправленный текст.

In [32]:
def spell_check(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    probability = 0
    #поиск ошибок
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    for i, trigrams_string in enumerate(trigram_list):
        word = text.split()[i]
        for trigram in trigrams_string.split():
            if not(trigram_wiki_model.get(trigram)) or \
            (trigram_wiki_model.get(trigram) < (10/8297222)):
                if ' ' + word + ' ' not in rucorpora_dict:
                    #print(word)
                    
                    #генерируем кандидатов + убираем невозможные по 3-грамной модели, оставшиеся проверяем по словарю
                    candidates = kill_candidates(word)
                    
                    #print(candidates)
                    if (candidates): #если множество оставшихся кандидатов не пустое
                        if(i > 0): #если слово не первое в тексте, находим левый контекст
                            left_context = text.split()[i-1]
                        if i + 1 < len(text.split()): # если слово не последнее в тексте
                            right_context = text.split()[i+1]
                        #определяем наиболее вероятного
                        cand , probability = select_best_candidate(candidates, left_context, right_context)
                        #print('Prob:', probability)
                        if probability == 0:
                            word = select_with_whitespace(cand)
                        else:
                            word = cand
        text_correct += ' ' + word
    return text_correct.strip()
                

In [33]:
train_df.wrong[159]

'усовершенстваванный алгоритм по сравнению с встроенным но работает только с английскими словами'

In [283]:
%%time
spell_check(train_df.wrong[159])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 679 µs


'усовершенстваванный алгоритм по сравнению с встроенным но работает только с английскими словами'

In [269]:
%%time
spell_check('какието пейзажы за окном незнакомые')

CPU times: user 24.9 s, sys: 88 ms, total: 25 s
Wall time: 25.9 s


'какието пейзажи за окном незнакомые'

In [271]:
search_errors(train_df.wrong[1])

['такойже']

## Ура! Хотя бы на одном предложении сработало

In [62]:
texts_correct = []
for text in tqdm(train_df.wrong.values[:15]):
    texts_correct.append(spell_check(text))

  0%|          | 0/15 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [193]:
print(train_df.wrong.values[:15])

['тоесть если дамой то только в том случае если потеряю работу в сентябре'
 'но если у меня характер будет такойже как хотя бы у одного из них я продам свою почку а на все вырученные деньги накуплю герлянд'
 'делать осьминогов из одуванчиков и куколок из шиповника'
 'вальс в па внимание будет под другую музыку с переходом местами на обычный вальс подробная схема будет выложена'
 'зима подходит к завершению а мне только сейчас удалось выбраться на каток'
 'ей в общем то все равно было на кого учиться лишь бы мама с папой не переживали что у них девочка не пристроена'
 'я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебрянной ложкой сахар добавить горячего молока чтоб кофе оставался еще дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день'
 'делая недюженные усилия над собой я таки выкарабкиваюсь из машины через заднюю дверь отползаю на обочину и видя как в сторону машины бегут ка

In [190]:
texts_correct

['тоесть если дамой то только в том случае если потеряю работу в сентябре',
 'но если у меня характер будет такойже как хотя бы у одного из них я продам свою почку а на все вырученные деньги накуплю герлянд',
 'делать осьминогов из одуванчиков и куколок из шиповника',
 'вальс в па внимание будет под другую музыку с переходом местами на обычный вальс подробная схема будет выложена',
 'зима подходит к завершению а мне только сейчас удалось выбраться на каток',
 'ей в общем то все равно было на кого учиться лишь бы мама с папой не переживали что у них девочка не пристроена',
 'я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебрянной ложкой сахар добавить горячего молока чтоб кофе оставался еще дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день',
 'делая недюжинные усилия над собой я таки выкарабкиваюсь из машины через заднюю дверь отползаю на обочину и видя как в сторону машины б

В первых 15 предложениях за почти 7 минут исправились 2 ошибки:
недюжинные, пейзажи

Остальные ошибки:
тоесть, такойже, тогдасоответственно, какието  - не могли исправить
милион, фамилие, конешно - думаю, что даже не нашли 

## TODO
1. Добавить генерацию кандидатов с пробелами
2. Убрать проверку кандидатов по словарю, сразу переходить к вероятностям (?)
3. Быстрый поиск по словарю.

In [23]:
def insert_whitespace(word):
    space = ' '
    cursor = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    insert = [L + l + R for l in space for (L,R) in cursor]
    return insert

In [24]:
def aprior_probability(word):
    word_aprior = 0
    s = sum(dict(wiki_freq).values())
    if dict(wiki_freq).get(word):
        word_aprior = dict(wiki_freq).get(word)/s
    else:
        word_aprior = 0
    return word_aprior

In [25]:
def select_with_whitespace(word):
    candidates = insert_whitespace(word)
    words = []
    probs = []
    for cand in candidates:
        cand = cand.split()
        if len(cand) == 2:
            probability = aprior_probability(cand[0])*aprior_probability(cand[1])
            words.append(' '.join(cand))
            probs.append(probability)
    cands = list(reversed(sorted(zip(words, probs), key=lambda x: x[1])))
#    final_word = ''. join([word[0] for word in cands if word[1]>0])
    
    return cands[0]

In [33]:
cand, prob = select_with_whitespace('какието')
print(cand, prob)

какие то 2.3022487100499806e-08


### Быстрый поиск

In [18]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()

In [19]:
keyword_processor.add_keywords_from_list(rucorpora_dict.split())

In [70]:
keyword_processor.extract_keywords('пейзаж')

['пейзаж']

In [73]:
keyword_processor.extract_keywords('вёдрами') # == ['ведрами']

['вёдрами']

In [46]:
def search_errors_dict(text):
    errors = []
    for word in text.split():
        if not keyword_processor.extract_keywords(word) == [word]:
            errors.append(word)
    return errors

In [47]:
all_errors = []
for text in tqdm(train_df.wrong):
    all_errors.extend(search_errors_dict(text))

100%|████████████████████████████████████| 3000/3000 [00:00<00:00, 3348.02it/s]


In [48]:
all_errors

['тоесть',
 'такойже',
 'герлянд',
 'серебрянной',
 'еще',
 'недюженные',
 'какието',
 'фамилие',
 'пейзажы',
 'самыи',
 'своеи',
 'милион',
 'тогдасоответственно',
 'конешно',
 'силно',
 'пян',
 'вспишка',
 'обектив',
 'черном',
 'написанно',
 'переконвертить',
 'музончик',
 'зачетный',
 'почемуто',
 'ее',
 'сво',
 'конешно',
 'щастье',
 'спатифиллум',
 'вобще',
 'никоглда',
 'отличаються',
 'хочеться',
 'четко',
 'пролжительность',
 'небльшая',
 'днем',
 'собстно',
 'учетом',
 'свое',
 'вобщем',
 'замерз',
 'сейцас',
 'еще',
 'ещее',
 'тыш',
 'расжился',
 'естесственно',
 'както',
 'подощрительно',
 'еще',
 'ее',
 'вообщем',
 'прошел',
 'разьве',
 'пескин',
 'серьезное',
 'универа',
 'ранво',
 'матану',
 'преподше',
 'замерзнуть',
 'клафилина',
 'ниче',
 'четвертый',
 'получаеться',
 'отпразную',
 'проостите',
 'милиардер',
 'меркле',
 'свое',
 'черный',
 'можо',
 'еще',
 'можнго',
 'живет',
 'переодически',
 'навверное',
 'вобщем',
 'когдато',
 'вообщем',
 'вкурсе',
 'нравяться',
 '

In [54]:
for i, text in enumerate(train_df.wrong):
    if 'еще' in text:
        print(i, text)

6 я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебрянной ложкой сахар добавить горячего молока чтоб кофе оставался еще дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день
38 лен буду попозже сейцас в душ пойду и зарядку еще хочу сделать все хочу и хочу не пока не делаю
40 ещее один вопрос волнует меня почему в одном из магазинов он стоит тыш когда везде
50 масс старт в биатлоне впереди еще например
70 еще можнго в бикини ходить зимой по системе иванова
82 это еще хорошо что ты в рыбном журнале работаешь
87 то есть размещение в пространстве таково что преподаватель ниже
98 приближается облако гнева не давайте ему приблизиться но еще на далеком расстоянии разворачиваете его и направляйте в другую сторону
160 чесно говоря это еще самая доброжелательная маска
176 надо всегда идти дальше если того требует сердце разум или что то там еще внутри еще раз удачи
257 мне еще тридцати не

1732 так что девчонки может еще есть шанс главное их не избаловать чтобы не испортились
1744 дома ждет еще одна книга подобного плана даже незнаю начать ее завтра читать или разбавить чем то романтическим или фанатастическим
1759 хоть водитель переодически открывал окно и было жутко холодно все же я мечтала о том что бы пробка подольше не розсасывалась и я могла еще подрыхнуть
1798 к сожалению летняя погода еще не устоялась поэтому большую часть нашего прибывания в пуконе шли дожди и наши дни ыглядели примерно так
1881 я согласна с таким положением вещей но
1890 прррроклятые буржуи будут морить меня незнанием результатов еще недели три минимум ближе кембриджа не нашлось никого согласного все это проверять конечно же
1903 более отвратительной организации тура я объехав более стран еще не встречала
2053 видно да еще когда темно и я блин весь как приведение в черном
2054 а впреди еще ооочень много разных социальных ролей
2071 по мнению автора инициативы финансовые векселя смогут использов

### Спел-чек без триграм

In [143]:
"""def spell_check_fast_dict(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    probability_1 = 0
    probability_2 = 0
    cand_1 = ''
    #поиск ошибок
    text_list = text.split()
    for index, word in enumerate(text_list):
        if not keyword_processor.extract_keywords(word) == [word]:
            candidates = kill_candidates(word)                 
        #print(candidates)
            if (candidates): #если множество оставшихся кандидатов не пустое
                if(index > 0): #если слово не первое в тексте, находим левый контекст
                    left_context = text_list[index-1]
                if index + 1 < len(text_list): # если слово не последнее в тексте
                    right_context = text_list[index+1]
                cand_1, probability_1 = select_best_candidate(candidates, 
                                                        left_context, 
                                                        right_context)
               # if probability:
                word = cand
           # if not(candidates): #or not(probability):
            cand_2, probability_2 = select_with_whitespace(word)
            
            if cand_1:
                
                if probability_2 > probability_1:
                    word = cand_2
                word = cand_1
            else:
                if probability_2:
                    word = cand_2
                                
        text_correct += ' ' + word
    return text_correct.strip()"""
                

In [38]:
def spell_check_fast_dict(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    #probability_1 = 0
    #probability_2 = 0
    #cand_1 = ''
    
    #поиск ошибок
    text_list = text.split()
    for index, word in enumerate(text_list):
        if not keyword_processor.extract_keywords(word) == [word]:
            
            cand_2, probability_2 = select_with_whitespace(word) #выбираем лучшего кандидата с пробелами
            
            candidates = kill_candidates(word) #генерируем кандидатов по edit distance + проверка по словарю               
            #print(candidates)
            if (candidates): #если множество оставшихся кандидатов не пустое
                if(index > 0): #если слово не первое в тексте, находим левый контекст
                    left_context = text_list[index-1]
                if index + 1 < len(text_list): # если слово не последнее в тексте
                    right_context = text_list[index+1]
                cand_1, probability_1 = select_best_candidate(candidates, left_context, right_context)
                
                if probability_2 > probability_1:
                    word = cand_2
                else:
                    word = cand_1
            
            else:
                if probability_2:
                    word = cand_2
                                
        text_correct += ' ' + word
    return text_correct.strip()
                

In [42]:
print(train_df.wrong.values[:2])

['тоесть если дамой то только в том случае если потеряю работу в сентябре'
 'но если у меня характер будет такойже как хотя бы у одного из них я продам свою почку а на все вырученные деньги накуплю герлянд']


In [114]:
keyword_processor.extract_keywords('серебряный')

['серебряный']

In [39]:
%%time
spell_check_fast_dict('герлянд много не бывает такойже')

Wall time: 4.53 s


'гирлянд много не бывает такой же'

In [58]:
print(train_df.wrong[6])

я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебрянной ложкой сахар добавить горячего молока чтоб кофе оставался еще дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день


In [57]:
%%time
spell_check_fast_dict(train_df.wrong[6])

Wall time: 6.07 s


'я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебряной ложкой сахар добавить горячего молока чтоб кофе оставался ещё дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день'

In [126]:
select_best_candidate(kill_candidates('тоесть'), '', 'если')

('тесть', 4.4086219419318365e-06)

In [37]:
aprior_probability('универ')

0

In [127]:
select_best_candidate(select_with_whitespace('тоесть')

('то есть', 4.6264235982909136e-07)

In [144]:
s = 'мамуля пришла из универ'
spell_check_fast_dict(s)

'мамуля пришла из унив ер'

In [136]:
select_with_whitespace('универ')

('унив ер', 1.2147467141801773e-12)

In [59]:
x = train_df.wrong[:500]

In [60]:
%%time
corrected = [spell_check_fast_dict(text) for text in tqdm(x)] 

100%|████████████████████████████████████████| 500/500 [23:31<00:00,  2.82s/it]


Wall time: 23min 31s


In [61]:
corrected[:20]

['тесть если дамой то только в том случае если потеряю работу в сентябре',
 'но если у меня характер будет такой же как хотя бы у одного из них я продам свою почку а на все вырученные деньги накуплю гирлянд',
 'делать осьминогов из одуванчиков и куколок из шиповника',
 'вальс в па внимание будет под другую музыку с переходом местами на обычный вальс подробная схема будет выложена',
 'зима подходит к завершению а мне только сейчас удалось выбраться на каток',
 'ей в общем то все равно было на кого учиться лишь бы мама с папой не переживали что у них девочка не пристроена',
 'я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебряной ложкой сахар добавить горячего молока чтоб кофе оставался ещё дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день',
 'делая недюжинные усилия над собой я таки выкарабкиваюсь из машины через заднюю дверь отползаю на обочину и видя как в сторону машины бе

In [154]:
train_small = train_df[:500]
train_small['corrected'] = corrected

/home/nst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [156]:
train_small.head()

,wrong,right,corrected
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...,тесть если дамой то только в том случае если п...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


In [160]:
errors = train_small[train_small.right != train_small.corrected]

In [163]:
errors

,wrong,right,corrected
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...,тесть если дамой то только в том случае если п...
6,я пью кофе не ради проснуться а ради процесса ...,я пью кофе не ради проснуться а ради процесса ...,я пью кофе не ради проснуться а ради процесса ...
7,делая недюженные усилия над собой я таки выкар...,делая недюжинные усилия над собой я таки выкар...,делая недюжинные усилия над собой я таки выкар...
8,ударение в фамилие уже наверно плавно переехал...,ударение в фамилии уже наверно плавно переехал...,ударение в фамилия уже наверно плавно переехал...
10,самыи прикол в том что делая дела в своеи жизн...,самый прикол в том что делая дела в своей жизн...,самых прикол в том что делая дела в своей жизн...
14,да там их мало конешно и я бил силно пян и всп...,да там их мало конечно и я был сильно пьян и в...,да там их мало конечно и я бил сильно ян и всп...
15,на черном фоне белый квадрат а в нем написанно...,на черном фоне белый квадрат а в нем написано ...,на чёрном фоне белый квадрат а в нем написано ...
18,парочка прошлогодних снимков с правнуком племе...,парочка прошлогодних снимков с правнуком племя...,парочка прошлогодних снимков с правнуком племе...
19,музончик конечно зачетный и почемуто до боли з...,музончик конечно зачетный и почему то до боли ...,музончик конечно заметный и заметный до боли з...
20,в связи с заявлениями тимошенко об ухудшении с...,в связи с заявлениями тимошенко об ухудшении с...,в связи с заявлениями тимошенко об ухудшении с...


In [101]:
search_errors_dict('герлянд много не бывает такойже')

['герлянд', 'такойже']

In [116]:
kill_candidates('герлянд')

['гирлянд', 'герлянц']

In [117]:
select_with_whitespace('мерседес')

('мерседе с', 0.0)

In [118]:
select_best_candidate(['мерседес', 'мерсеес'], 'машина', 'бэнс')

('мерседес', 2.7553887137073974e-06)

In [104]:
select_best_candidate(['гирлянд', 'герлянц'], '', 'много')

('гирлянд', 0)